Since our data is time series, we must:
    1. Perform 31-day sliding window min-max scaling
    2. Create a 0-1 binary predictor variables on whether the coin was pumped
For __OHLCV__ Data

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 200)
%load_ext autoreload
%matplotlib inline

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
print(os.getcwd())

/Users/hn/Documents/USC Others/ISI Research/PumpCoin/CrytoCompare/All_Pump_Analysis/Before_Announcement_Analysis


# PART 1: SLIDING WINDOW NORMALIZATION

### GENERAL PROCEDURE FOR WINDOW NORMALIZATION
1. Iterate through row
2. For each of the group of variables (x_d0 to x_d31), apply the function
3. This returns an aray, keep appending them until we have the entire array
4. Append the un-scaled features, and now have a complete row
5. Keep stackign them to a blank array

## MIN-MAX NORMALIZATION

In [4]:
def minmax(x, varlist):
    x_t = x.T
    scaler = MinMaxScaler()
    scaler.fit(x_t)
    r = scaler.transform(x_t)
    return pd.DataFrame(r.T, columns=varlist)

In [5]:
#create a list of features
ohlcv_feat = pd.read_pickle("ohlcv_features_h.pkl")
len(ohlcv_feat)

245

In [6]:
orig_cols = list(d.columns)
d_scaled = d[cd['core']]
d_temp = np.zeros(shape=(1,d.shape[1]))
print("d_temp", d_temp.shape)

for k,v in cd.items():
    if k != 'core':
        var = cd[k] 
        d_temp = minmax(d[var], var)
        print(d_temp.shape, type(d_temp))
        d_scaled = pd.concat((d_scaled,d_temp),axis=1)

NameError: name 'd' is not defined

In [ ]:
#Verify that the scaled dataset has the same shape as the original
d_scaled.shape

In [ ]:
d_scaled['pumped_yn'] = 0
d_scaled.loc[d_scaled.currency == d_scaled.coin_name, 'pumped_yn'] = 1

###  PLOT THE VARIABLE GROUPS TO MAKE SURE THE SCALED ARE THE SAME
Check that the plots are essential the same, just with range normalized between and 0 and 1

In [ ]:
cnt = 0
#Select a random show
r = d_scaled[d_scaled.pumped_yn == 1].sample(1)
# ix = np.random.randint(0,high=5e4,size=1,dtype=int)
# s = d.loc[ix]
# s2 = d_scaled.loc[ix]
s = d.loc[r.index]
s2 = r
fig = plt.figure(figsize=(10,70))
print(r.currency)

for k,v in cd.items():
    if k != 'core':
        var = cd[k]
        ax1 = fig.add_subplot(33,2,cnt*2+1)
        ax1 = s.iloc[0][var].plot()
        ax1.set_xticklabels([])
        ax1.set_xlabel(k)
        ax2 =fig.add_subplot(33,2,cnt*2+2)
        ax2 = s2.iloc[0][var].plot()
        ax2.set_xticklabels([])
        ax2.set_xlabel(k)
        cnt+=1
r

In [ ]:
d_scaled.head()

In [ ]:
d_scaled.head(10)[['pumptime_edited','currency','exchange','coin_name','pumped_yn']]

In [ ]:
d_scaled.sort_values(['exchange','coin_name','timestamp']).head(10)

In [ ]:
# SAVE THE FILE
d_scaled.to_pickle("ohlcv_data_slide_scaled_h.pkl")